# Capstone Notebook

This notebook will contain all work for the Applied Data Science Capstone course

In [1]:
import pandas as pd
import numpy as np
import requests
import random

!pip install geopy
from geopy.geocoders import Nominatim

from pandas import json_normalize

!pip install folium
import folium

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Introduction/Business Problem
If you are looking to open a Cafe in Canada and sell coffee as an entrepreneur, you will be facing tough competition in a potentially saturated market. Therefore a key problem to solve is:

**Where is the best place to open up a Cafe?**

Solving this problem gives us information and prevents opening a Cafe in a non-saturated area may provide a new business the best opportunity to thrive without competition. This is a very difficult question to solve and another issue arises:

**How do we measure/determine which is the best place to start a Cafe?**

Data science and exploration of location data will be key to finding the solution.

## Data
To solve the problem, we will need the following data:


*   List of neighbourhoods in Canada. This defines the scope of the project to the city of Sydney. This data can be sourced from webscraping Wikipedia pages and or some location data service.
*   Latitude and longitude coordinates of these neighbourhoods for visualisation, clustering and other purposes. This data can be sourced from location data services.
*   Venue data for finding saturated neighbourhoods of Cafes. This data can be sourced from Foursquare's API.

The culmination of this data will provide insights into saturation of venues in specific neighbourhoods and potential vacancies in locations.



# Methodology
### List of neighbourhoods in Canada


In [3]:
df=pd.read_html("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050")[0]

In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
df = df[df.Borough != 'Not assigned']
df['Neighbourhood']=df['Neighbourhood'].replace('Not assigned', df['Borough'])
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Geospatial data:

In [6]:
!wget -q -O 'geospatial_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [7]:
df_geo = pd.read_csv('geospatial_data.csv')
df_geo.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [11]:
df_grouped = df.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(', '.join)
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [12]:
df_geo = df_geo.rename({'Postal Code':'Postcode'}, axis=1)

In [13]:
df2 = pd.merge(df_grouped,df_geo)

In [14]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


### Venue Data


In [15]:
# Foursquare Credentials
CLIENT_ID = 'FF0YXBOX3Y2E0QICY4DW3LTM5IP0CEL3EHIFRMQZBVZU0UK0' # your Foursquare ID
CLIENT_SECRET = 'YJO42BH315TOCCT3LODT1CMB20YLUNO14OMPQ2JBNE5CEAQN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: FF0YXBOX3Y2E0QICY4DW3LTM5IP0CEL3EHIFRMQZBVZU0UK0
CLIENT_SECRET:YJO42BH315TOCCT3LODT1CMB20YLUNO14OMPQ2JBNE5CEAQN


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Found {} venues in {} neighborhoods.'.format(nearby_venues.shape[0], len(venues_list)))
    
    return(nearby_venues)

In [17]:
venues = getNearbyVenues(names=df2['Neighbourhood'],
                         latitudes=df2['Latitude'],
                         longitudes=df2['Longitude'])

Found 2163 venues in 103 neighborhoods.


In [20]:
print(venues.shape)
venues.head()

(2163, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [23]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",97,97,97,97,97,97
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",9,9,9,9,9,9
"Alderwood, Long Branch",7,7,7,7,7,7
...,...,...,...,...,...,...
Willowdale West,7,7,7,7,7,7
Woburn,4,4,4,4,4,4
"Woodbine Gardens, Parkview Hill",12,12,12,12,12,12


# Exploring the Foursquare Data
We can use the 'Venue Category' field in the data to find what our venue of interest is referred to:

In [24]:
venues['Venue Category'].unique()

array(['Park', 'Bus Stop', 'Food & Drink Shop', 'Hockey Arena',
       'Portuguese Restaurant', 'Coffee Shop', 'Pizza Place', 'Bakery',
       'Distribution Center', 'Spa', 'Restaurant', 'Pub',
       'Breakfast Spot', 'Gym / Fitness Center', 'Historic Site',
       'Farmers Market', 'Dessert Shop', 'Chocolate Shop',
       'Performing Arts Venue', 'French Restaurant', 'Mexican Restaurant',
       'Café', 'Yoga Studio', 'Theater', 'Event Space',
       'Asian Restaurant', 'Shoe Store', 'Ice Cream Shop',
       'Electronics Store', 'Art Gallery', 'Cosmetics Shop', 'Bank',
       'Beer Store', 'Health Food Store', 'Antique Shop', 'Boutique',
       'Furniture / Home Store', 'Vietnamese Restaurant',
       'Clothing Store', 'Accessories Store', "Women's Store",
       'Miscellaneous Shop', 'Italian Restaurant', 'Creperie',
       'Sushi Restaurant', 'Arts & Crafts Store', 'Burrito Place',
       'Beer Bar', 'Hobby Shop', 'Diner', 'Fried Chicken Joint',
       'Smoothie Shop', 'Sandwich Pl

We find that Cafes are referred to as "Coffee Shop" and that there are 184 Cafe Venues in the Foursquare data.

In [31]:
venues[venues['Venue Category']=="Coffee Shop"].shape

(184, 7)

Let's visualise the locations on the map

In [34]:
cafe_data = venues[venues['Venue Category']=="Coffee Shop"]
cafe_data.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
9,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
21,Harbourfront,43.654260,-79.360636,Arvo,43.649963,-79.361442,Coffee Shop
23,Harbourfront,43.654260,-79.360636,Rooster Coffee,43.651900,-79.365609,Coffee Shop
25,Harbourfront,43.654260,-79.360636,Starbucks,43.651613,-79.364917,Coffee Shop


In [35]:
latitude = 43.6532
longitude = -79.3832

In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(cafe_data['Venue Latitude'], cafe_data['Venue Longitude'], cafe_data['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto